# LLM 推理和解码策略

> 解码策略决定了如何从模型输出的词汇表概率分布中选择下一个 token。不同的策略在生成文本的多样性、准确性和计算成本之间做出了不同的权衡。

## Greedy Search（贪心搜索）

贪心搜索是最简单直接的解码策略。在每个时间步，它都会选择当前概率最高的 token 作为输出，然后将这个 token 作为下个时间步的输入，继续生成。

- 优点：
  - 实现简单，计算速度快
- 缺点：
  - 容易陷入局部最优。在某个时间步选择的局部最优 token，可能会导致后续整个序列的质量下降
  - 生成的文本缺乏多样性，往往是重复和确定性

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def greedy_search(model_logits, max_len=20, eos_token_id=2):
    '''
    Args:
        model_logits (torch.Tensor): shape: (batch_size, seq_len, vocab_size)
        max_len (int): 最大生成长度
        eos_token_id (int): id of end-of-sentence token

    Returns:
        torch.Tensor: 生成的 token 序列
    '''
    batch_size = model_logits.size(0)

    # 存储生成的 token 索引
    generated_sequence = torch.zeros(batch_size, max_len, dtype=torch.long)

    # 模拟逐个 token 生成过程
    for t in range(max_len):
        
        # 获取当前时间步的 logits
        current_logits = model_logits[:, t, :]

        # 计算概率分布
        probs = F.softmax(current_logits, dim=-1)

        # 选择概率最高的 token 索引
        next_token = torch.argnax(probs, dim=-1)

        # 将选择的词加入生成序列中
        generated_sequence[:, t] = next_token

        # 检查所有 batch 都遇到结束标记，遇到则提前停止
        if (next_token == eos_token_id).all():
            break
    
    return generated_sequence

## Beam Search

束搜索是对贪心搜索的一种改进，它在一定程度上克服了局部最优的问题。

- 核心思想：在每个时间步，保留一个束宽 `beam_size` 个概率最高的候选序列。在下一个时间步，会基于这 `beam_size` 个候选序列，分别生成下一个词，然后从所有可能的序列中，再次选出总概率最高的 `beam_size` 个，并不断重复这个过程。
- 优点：生成序列质量比贪心搜索更高，它考虑了更广的搜索空间。
- 缺点：
  - 计算成本更高，是贪心搜索的 `beam_size` 倍。
  - 仍然可能错过全局最优解。
  - 生成的文本可能偏向高频、安全的短语，多样性依然有限。

In [ ]:
import torch

def beam_search(lm_prob, beam_size=3):
    '''
    Args:
        lm_probs (torch.Tensor): 模型输出的概率张量，shape: (batch, seq_len, vocab_size)
        beam_size (int): 束宽

    Returns:
        tuple: (序列索引，对应的对数概率)
    '''

    batch, seq_len, vocab_size = lm_prob.shape
    
    # 为了避免下溢出并且将连乘转化为连加，对概率取对数
    log_lm_prob = torch.log(lm_prob)

    # -- initalization --
    # 取第一个时间步概率最高的 k 个 token 作为初始 beam
    # shape: log_beam_prob: (batch, beam_size) indices: (batch, beam_size)
    log_beam_prob, indices = log_lm_prob[:, 0, :].topk(beam_size, sorted=True)

    # 将 indices 扩展一维，用于后续拼接
    # indices: (batch, beam_size, 1)
    indices = indices.unsqueeze(-1)

    # 逐时间步扩展 Beam
    for i in range(1, seq_len):
        # 1. 扩展所有候选
        # log_beam_prob: (batch, beam_size) -> (batch, beam_size, 1)
        # log_lm_prob: (batch, vocab_size) -> (batch, 1, vocab_size)
        # current_log_probs: (batch, beam_size, vocab_size)
        current_log_probs = log_beam_prob.unsqueeze(-1) + log_lm_prob.unsqueeze(1)
        # 2. 选取 top-k
        # 将 beam_size 和 vocab_size 维度合并，方便选取 top-k
        # current_log_probs: (batch, beam_size * vocab_size)
        current_log_probs = current_log_probs.view(batch, -1)
        log_beam_prob, topk_indices = current_log_probs.topk(beam_size, sorted=True)
        # 3. 更新 indices
        # 计算对应的 beam 索引和 token 索引
        beam_indices = topk_indices // vocab_size  # (batch, beam_size)
        token_indices = topk_indices % vocab_size  # (batch, beam_size)
        # 根据 beam_indices 从之前的 indices 中选取对应的序列
        # indices: (batch, beam_size, i)
        selected_indices = torch.gather(indices, 1, beam_indices.unsqueeze(-1).expand(-1, -1, i))
        # 拼接新的 token 索引
        # indices: (batch, beam_size, i + 1)
        indices = torch.cat([selected_indices, token_indices.unsqueeze(-1)], dim=-1)
    return indices, log_beam_prob